# Angular correlation of bright object in LSST

In [1]:
import astropy.units as u
import numpy as np 
import pandas as pd
pd.set_option("display.max_columns", None)
from astropy.time import Time

import scipy.stats

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus


import lsst.geom as geom
import pickle
from astropy.stats import SigmaClip

In [2]:
from astropy.io import fits
from astropy.wcs import WCS


import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

import scipy
from scipy.optimize import curve_fit

In [3]:
df = pd.read_csv("list_of_isolatedobjects.csv",index_col=0)

In [4]:
df

,idx_obj,ra,dec,magG,magR,magI,magZ,magY,smagG,smagR,smagI,smagZ,smagY,G_R,Z_Y,racosdec
0,60.0,122.458941,-36.203409,15.780089,15.296876,15.066548,14.977860,14.996176,0.017255,0.017876,0.018660,0.019977,0.017382,0.483214,-0.088688,98.815202
1,97.0,122.443234,-36.188544,16.447322,15.196784,14.543295,14.210779,14.027632,0.027123,0.020171,0.025437,0.018415,0.032334,1.250538,-0.332516,98.821288
2,101.0,122.476591,-36.168954,13.856892,13.132240,12.887973,12.778402,12.795855,0.023862,0.019996,0.022695,0.020973,0.033263,0.724652,-0.109571,98.872928
3,106.0,122.558799,-36.184124,14.714332,13.661116,13.156095,12.924743,12.855121,0.024281,0.020753,0.017012,0.022227,0.035247,1.053215,-0.231352,98.920140
4,114.0,122.556550,-36.138403,15.643555,15.108538,14.877074,14.796453,14.863543,0.021742,0.016813,0.019532,0.024336,0.048665,0.535016,-0.080621,98.976031
5,120.0,122.565977,-36.177767,15.710948,15.322174,15.119611,15.009051,15.005122,0.023673,0.019078,0.022171,0.024533,0.033143,0.388774,-0.110560,98.933961
6,121.0,122.467500,-36.198182,15.857151,15.312018,15.046633,14.935572,14.934684,0.019609,0.017451,0.018923,0.020449,0.033245,0.545134,-0.111061,98.828706
7,131.0,122.573564,-36.178004,14.344957,14.062978,13.957407,13.958046,14.074584,0.020347,0.022226,0.019226,0.023528,0.040077,0.281978,0.000638,98.939786
8,132.0,122.532131,-36.209821,15.419324,14.167039,13.591124,13.306499,13.189422,0.021029,0.018542,0.017178,0.017337,0.035554,1.252285,-0.284624,98.866161
9,151.0,122.562104,-36.210801,15.840656,15.212063,14.888323,14.744011,14.755129,0.020431,0.019651,0.024061,0.020587,0.039148,0.628592,-0.144311,98.889106
